In [ ]:
import pickle

In [ ]:
rets = {}

In [ ]:
%ls

In [ ]:
path = "policy_dissection_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [ ]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [ ]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [ ]:
print(averaged_ret[1])